In [1]:
from src import processing, utils

import os
import numpy as np
import pandas as pd
from   glob  import glob
from   re    import sub

# Extract NIFTY images from compressed files

## Extract absolute Jacobian images

In [2]:
# Directory containing compressed images
input_dir_abs = 'data/human/registration/jacobians/absolute/smooth/'
input_dir_abs = os.path.join(input_dir_abs, '')

# List of paths to compressed images
input_files_abs = glob(input_dir_abs+'*.gz')

# Extract absolute jacobians
# unzipped_files_abs = processing.gunzip_files(infiles = input_files_abs, 
#                                              keep = True,
#                                              parallel = True,
#                                              nproc = 8)

unzipped_files_abs = glob(input_dir_abs+'*.nii')
unzipped_files_abs[:5]

['data/human/registration/jacobians/absolute/smooth/d8_0096_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/12013_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/21045_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/d8_0673_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/absolute/smooth/sub-1050792_ses-01_T1w.extracted_fwhm_4vox.nii']

## Extract relative Jacobian images

In [3]:
# Directory containing compressed images
input_dir_rel = 'data/human/registration/jacobians/relative/smooth/'
input_dir_rel = os.path.join(input_dir_rel, '')

# List of paths to compressed images
input_files_rel = glob(input_dir_rel+'*.gz')

# Extract relative jacobians
# unzipped_files_rel = processing.gunzip_files(infiles = input_files_rel, 
#                                              keep = True,
#                                              parallel = True,
#                                              nproc = 8)

unzipped_files_rel = glob(input_dir_rel+'*.nii')
unzipped_files_rel[:5]

['data/human/registration/jacobians/relative/smooth/d8_0773_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/sub-1050249_ses-01_T1w.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/d8_0214_01.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/21025_T1.extracted_fwhm_4vox.nii',
 'data/human/registration/jacobians/relative/smooth/sub-3940059_ses-01_run-02_T1w.extracted_fwhm_4vox.nii']

---

# Convert NIFTY images to MINC

## Convert absolute Jacobian images

In [4]:
# Directory in which to save the converted images
outdir_abs = 'data/human/registration/jacobians/absolute/smooth_minc/'

# Convert absolute jacobian images to MINC
# imgfiles_abs = processing.convert_images(infiles = unzipped_files_abs,
#                                          input_format = 'nifty',
#                                          output_format = 'minc',
#                                          outdir = outdir_abs,
#                                          keep = True,
#                                          parallel = True,
#                                          nproc = 8)

imgfiles_abs = glob(outdir_abs+'*.mnc')
imgfiles_abs[:5]

['data/human/registration/jacobians/absolute/smooth_minc/sub-1140440_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050100_ses-01_run-02_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/d8_0285_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/sub-1050192_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/absolute/smooth_minc/41014_T1.extracted_fwhm_4vox.mnc']

## Convert relative Jacobian images

In [5]:
# Directory in which to save the converted images
outdir_rel = 'data/human/registration/jacobians/relative/smooth_minc/'

# Convert absolute jacobian images to MINC
# imgfiles_rel = processing.convert_images(infiles = unzipped_files_rel,
#                                          input_format = 'nifty',
#                                          output_format = 'minc',
#                                          outdir = outdir_rel,
#                                          keep = True,
#                                          parallel = True,
#                                          nproc = 8)

imgfiles_rel = glob(outdir_rel+'*.mnc')
imgfiles_rel[:5]

['data/human/registration/jacobians/relative/smooth_minc/32014_T1.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/sub-3940197_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/d8_0219_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/d8_0092_01.extracted_fwhm_4vox.mnc',
 'data/human/registration/jacobians/relative/smooth_minc/11034_T1.extracted_fwhm_4vox.mnc']

---

# Specify which cohorts to use

In [6]:
#Create directory to save data subset derivatives
human_basedir = 'data/human/derivatives'
datasets = ['POND', 'SickKids']    
human_basedir = utils.mkdir_from_list(inlist = datasets,
                                      basedir = human_basedir)
human_basedir

'data/human/derivatives/POND_SickKids'

## Filter demographics for cohorts of interest

In [7]:
# Path to the demographics file 
demographics = 'data/human/registration/DBM_input_demo_passedqc.csv'

#Import demographics
df_demographics = pd.read_csv(demographics)

#Filter individuals for data subset
df_demographics = (df_demographics
                   .loc[df_demographics['Dataset'].isin(datasets)]
                   .copy())

#Create a new column for patient image files
df_demographics['File'] = (df_demographics['Extract_ID']
                           .str.replace('.mnc', 
                                        '_fwhm_4vox.mnc', 
                                        regex = True))

#Write out demographics subset to subset directory
demographics = os.path.join(human_basedir, os.path.basename(demographics))
df_demographics.to_csv(demographics, index = False)

## Create symlinks to absolute jacobian images

In [8]:
#Create output directory for absolute jacobian images
imgdir_abs = os.path.join(human_basedir, 'jacobians', 'absolute', '')

#Get image file names
imgfiles_abs_dataset = df_demographics['File'].to_list()

#Get image paths for images in the subset
imgfiles_abs_dataset = [[f for f in imgfiles_abs if g in f][0] 
                        for g in imgfiles_abs_dataset]

#Create symbolic links for subset images to the subset directory
imgfiles_abs = utils.mk_symlinks(src = imgfiles_abs_dataset,
                                 dst = imgdir_abs)
imgfiles_abs[:5]

['data/human/derivatives/POND_SickKids/jacobians/absolute/sub-1050004_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/absolute/sub-1050027_ses-01_run-03_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/absolute/sub-1050032_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/absolute/sub-1050039_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/absolute/sub-1050040_ses-01_T1w.extracted_fwhm_4vox.mnc']

## Create symlinks to relative jacobian images

In [9]:
#Create output directory for absolute jacobian images
imgdir_rel = os.path.join(human_basedir, 'jacobians', 'relative', '')

#Get image file names
imgfiles_rel_dataset = df_demographics['File'].to_list()

#Get image paths for images in the subset
imgfiles_rel_dataset = [[f for f in imgfiles_rel if g in f][0] 
                        for g in imgfiles_rel_dataset]

#Create symbolic links for subset images to the subset directory
imgfiles_rel = utils.mk_symlinks(src = imgfiles_rel_dataset,
                                 dst = imgdir_rel)
imgfiles_rel[:5]

['data/human/derivatives/POND_SickKids/jacobians/relative/sub-1050004_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/relative/sub-1050027_ses-01_run-03_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/relative/sub-1050032_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/relative/sub-1050039_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/jacobians/relative/sub-1050040_ses-01_T1w.extracted_fwhm_4vox.mnc']

---

# Calculating human participant effect sizes

In [10]:
# Path to the mask image to use
maskfile = 'data/human/registration/reference_files/mask.mnc'

# Number of controls to use for propensity matching
ncontrols = 10

## Absolute jacobian effect sizes

In [11]:
#Output directory
es_dir_abs = os.path.join(human_basedir, 
                          'effect_sizes', 
                          'absolute',
                          'resolution_0.5_ncontrols_{}'.format(ncontrols),
                          '')
es_dir_abs

'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/'

In [12]:
# Calculate effect size images using absolute Jacobians
# es_files_abs = processing.calculate_human_effect_sizes(demographics = demographics,
#                                                        imgdir = imgdir_abs,
#                                                        maskfile = maskfile,
#                                                        outdir = es_dir_abs, 
#                                                        ncontrols = ncontrols,
#                                                        parallel = True,
#                                                        nproc = 4)

es_files_abs = glob(es_dir_abs+'*.mnc')
es_files_abs[:5]

['data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/sub-3940220_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/sub-0880650_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/sub-1050633_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/d8_0041_03.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_0.5_ncontrols_10/sub-0880409_ses-01_T1w.extracted_fwhm_4vox.mnc']

## Relative jacobian effect sizes

In [13]:
#Output directory
es_dir_rel = os.path.join(human_basedir, 
                          'effect_sizes', 
                          'relative',
                          'resolution_0.5_ncontrols_{}'.format(ncontrols),
                          '')
es_dir_rel

'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/'

In [14]:
# Calculate effect size images using relative Jacobians
# es_files_rel = processing.calculate_human_effect_sizes(demographics = demographics,
#                                                        imgdir = imgdir_rel,
#                                                        maskfile = maskfile,
#                                                        outdir = es_dir_rel, 
#                                                        ncontrols = ncontrols,
#                                                        parallel = True,
#                                                        nproc = 4)

es_files_rel = glob(es_dir_rel+'*.mnc')
es_files_rel[:5]

['data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/sub-0880669_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/sub-1050697_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/sub-1050398_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/sub-1140319_ses-01_run-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_0.5_ncontrols_10/sub-0881203_ses-01_T1w.extracted_fwhm_4vox.mnc']

# Downsampling the effect size images

## Downsampling absolute effect size images to 3.0mm

In [15]:
# Resolution (mm) to which we want to downsample
isostep = 3.0

In [16]:
# Create a separate output directory for downsampled images
es_dir_abs_downsampled_3mm = sub(r'resolution_0.5',
                                 'resolution_{}'.format(isostep),
                                 es_dir_abs)

# Downsample absolute effect size images
# es_files_abs_downsampled_3mm = processing.resample_images(infiles = es_files_abs,
#                                                           isostep = isostep,
#                                                           outdir = es_dir_abs_downsampled_3mm,
#                                                           parallel = True,
#                                                           nproc = 8)

es_files_abs_downsampled_3mm = glob(es_dir_abs_downsampled_3mm+'*.mnc')
es_files_abs_downsampled_3mm[:5]

['data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_3.0_ncontrols_10/sub-1050401_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_3.0_ncontrols_10/sub-0881008_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_3.0_ncontrols_10/sub-1050189_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_3.0_ncontrols_10/sub-1050284_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_3.0_ncontrols_10/sub-0881012_ses-01_T1w.extracted_fwhm_4vox.mnc']

## Downsampling relative effect size images to 3.0mm

In [17]:
# Create a separate output directory for downsampled images
es_dir_rel_downsampled_3mm = sub(r'resolution_0.5',
                                 'resolution_{}'.format(isostep),
                                 es_dir_rel)

# Downsample relative effect size images
# es_files_rel_downsampled_3mm = processing.resample_images(infiles = es_files_rel,
#                                                           isostep = isostep,
#                                                           outdir = es_dir_rel_downsampled_3mm,
#                                                           parallel = True,
#                                                           nproc = 8)

es_files_rel_downsampled_3mm = glob(es_dir_rel_downsampled_3mm+'*.mnc')
es_files_rel_downsampled_3mm[:5]

['data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_3.0_ncontrols_10/d8_0028_03.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_3.0_ncontrols_10/sub-3940257_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_3.0_ncontrols_10/sub-0880250_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_3.0_ncontrols_10/sub-0881189_ses-01_run-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_3.0_ncontrols_10/sub-1050229_ses-01_T1w.extracted_fwhm_4vox.mnc']

# Downsampling auxiliary imaging files

In [34]:
# Path to the neuroanatomical template image
model = 'data/human/registration/reference_files/model.mnc'

# Path to the mask image
mask = 'data/human/registration/reference_files/mask.mnc'

# # Downsample the template to 3.0 mm
# model_3mm = processing.resample_image(infile = model,
#                                       isostep = 3.0,
#                                       suffix = '_3.0mm')

# # Downsample the mask to 3.0 mm
# mask_3mm = processing.resample_image(infile = mask,
#                                      isostep = 3.0,
#                                      suffix = '_3.0mm')

---

# Clustering the participants

## Creating effect size matrices

### Absolute effect size matrix

In [19]:
# File in which to save the absolute effect size matrix
es_abs_matrix_csv = 'ES_jacobians_absolute_resolution_{}_ncontrols_{}.csv'.format(isostep, ncontrols)
es_abs_matrix_csv = os.path.join(es_dir_abs_downsampled_3mm, es_abs_matrix_csv)

# # Compute the absolute effect size matrix
# df_es_abs = processing.build_voxel_matrix(infiles = es_files_abs_downsampled_3mm,
#                                           mask = mask_3mm,
#                                           sort = True,
#                                           file_col = True,
#                                           parallel = False,
#                                           nproc = 2)

# # Extract file base names from paths
# df_es_abs['file'] = [os.path.basename(file) for file in df_es_abs['file']]

# # Write to file
# df_es_abs.to_csv(es_abs_matrix_csv, index = False)

### Relative effect size matrix

In [20]:
# File in which to save the relative effect size matrix
es_rel_matrix_csv = 'ES_jacobians_relative_resolution_{}_ncontrols_{}.csv'.format(isostep, ncontrols)
es_rel_matrix_csv = os.path.join(es_dir_rel_downsampled_3mm, es_rel_matrix_csv)

# # Compute the relative effect size matrix
# df_es_rel = processing.build_voxel_matrix(infiles = es_files_rel_downsampled_3mm,
#                                           mask = mask_3mm,
#                                           sort = True,
#                                           file_col = True,
#                                           parallel = False,
#                                           nproc = 2)

# # Extract file base names from paths
# df_es_rel['file'] = [os.path.basename(file) for file in df_es_rel['file']]

# # Write to file
# df_es_rel.to_csv(es_rel_matrix_csv, index = False)

# Clustering the participants

In [21]:
infiles = [es_abs_matrix_csv, es_rel_matrix_csv]
nk_max = 10
metric = 'correlation'
K = 10
sigma = 0.5
t = 20

cluster_basedir = os.path.join(human_basedir, 'clusters')
cluster_params = {'es_ncontrols':ncontrols,
                 'clust_resolution':isostep,
                 'clust_nk_max':nk_max,
                 'clust_metric':metric,
                 'clust_K':K,
                 'clust_sigma':sigma,
                 'clust_t':t}

(cluster_dir, cluster_metadata) = utils.mkdir_from_params(params = cluster_params,
                                                            basedir = cluster_basedir,
                                                            return_metadata = True)

cluster_file = os.path.join(cluster_dir, 'human_clusters.csv')

In [22]:
# df_clusters = processing.cluster_human_data(infiles = infiles,
#                                             rownames = 'file',
#                                             nk_max = nk_max,
#                                             metric = metric,
#                                             K = K,
#                                             sigma = sigma,
#                                             t = t,
#                                             cluster_file = cluster_file)

# df_clusters.head()

Importing data...
Running similarity network fusion...
Assigning clusters...


,ID,nk2,nk3,nk4,nk5,nk6,nk7,nk8,nk9,nk10
0,d8_0001_04.extracted_fwhm_4vox.mnc,1,1,2,2,3,6,8,8,5
1,d8_0002_03.extracted_fwhm_4vox.mnc,1,1,1,1,1,7,3,2,9
2,d8_0003_02.extracted_fwhm_4vox.mnc,1,1,2,5,5,2,5,6,10
3,d8_0004_01.extracted_fwhm_4vox.mnc,1,1,1,2,3,6,8,8,5
4,d8_0006_03.extracted_fwhm_4vox.mnc,1,1,2,5,5,2,5,6,10


**To-do**: Implement W-matrix in clustering

# Create cluster maps

## Downsampling absolute effect size images to 1.0mm

In [27]:
# Resolution (mm) to which we want to downsample
isostep = 1.0

In [31]:
# Create a separate output directory for downsampled images
es_dir_abs_1mm = sub(r'resolution_0.5',
                     'resolution_{}'.format(isostep),
                     es_dir_abs)

# Downsample absolute effect size images
es_files_abs_1mm = processing.resample_images(infiles = es_files_abs,
                                              isostep = isostep,
                                              outdir = es_dir_abs_1mm,
                                              parallel = True,
                                              nproc = 8)

es_files_abs_1mm = glob(es_dir_abs_1mm+'*.mnc')
es_files_abs_1mm[:5]

100%|█████████████████████████████████████████████████████████████████████████████████████| 424/424 [10:13<00:00,  1.45s/it]


['data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_1.0_ncontrols_10/sub-1050395_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_1.0_ncontrols_10/sub-1050724_ses-01_run-02_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_1.0_ncontrols_10/sub-3940119_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_1.0_ncontrols_10/d8_0335_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/absolute/resolution_1.0_ncontrols_10/sub-3940340_ses-01_T1w.extracted_fwhm_4vox.mnc']

## Downsampling relative effect size images to 3.0mm

In [32]:
# Create a separate output directory for downsampled images
es_dir_rel_1mm = sub(r'resolution_0.5',
                     'resolution_{}'.format(isostep),
                     es_dir_rel)

# Downsample relative effect size images
es_files_rel_1mm = processing.resample_images(infiles = es_files_rel,
                                              isostep = isostep,
                                              outdir = es_dir_rel_1mm,
                                              parallel = True,
                                              nproc = 8)

es_files_rel_1mm = glob(es_dir_rel_1mm+'*.mnc')
es_files_rel_1mm[:5]

100%|█████████████████████████████████████████████████████████████████████████████████████| 424/424 [10:15<00:00,  1.45s/it]


['data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_1.0_ncontrols_10/sub-1050633_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_1.0_ncontrols_10/d8_0209_01.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_1.0_ncontrols_10/sub-0880650_ses-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_1.0_ncontrols_10/sub-1050826_ses-01_run-01_T1w.extracted_fwhm_4vox.mnc',
 'data/human/derivatives/POND_SickKids/effect_sizes/relative/resolution_1.0_ncontrols_10/sub-0880136_ses-01_run-02_T1w.extracted_fwhm_4vox.mnc']

# Downsampling auxiliary imaging files

In [35]:
# Downsample the template to 3.0 mm
model_1mm = processing.resample_image(infile = model,
                                      isostep = 1.0,
                                      suffix = '_1.0mm')

# Downsample the mask to 3.0 mm
mask_1mm = processing.resample_image(infile = mask,
                                     isostep = 1.0,
                                     suffix = '_1.0mm')

## Create absolute cluster maps

In [46]:
df_cluster_metadata  = pd.read_csv(cluster_metadata)
df_cluster_params = pd.DataFrame(cluster_params, index = [0])
df_cluster_params = pd.merge(df_cluster_metadata, df_cluster_params, how = 'inner')
cluster_id = str(df_cluster_params['id'].values[0])

cluster_map_params = cluster_params.copy()
cluster_map_params['clust_map_method'] = 'mean'

cluster_map_id = '-'.join([cluster_id, utils.random_id(3)])

cluster_map_basedir = os.path.join(human_basedir, 'cluster_maps')
cluster_map_basedir_abs = os.path.join(cluster_map_basedir, 'absolute')

(cluster_map_dir_abs, metadata_cluster_map) = utils.mkdir_from_params(params = cluster_map_params, 
                                                                      basedir = cluster_map_basedir_abs,
                                                                      params_id = cluster_map_id,
                                                                      return_metadata = True)
cluster_map_dir_abs

'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/'

In [39]:
# cluster_maps_abs = processing.create_cluster_maps(clusters = cluster_file,
#                                                   imgdir = es_dir_abs_1mm,
#                                                   outdir = cluster_map_dir_abs,
#                                                   method = 'mean',
#                                                   mask = mask_1mm)

cluster_maps_abs = glob(cluster_map_dir_abs+'*.mnc')
cluster_maps_abs[:5]

['data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_9_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_4_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_5_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_10_k_8_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/absolute/380-581/cluster_map_nk_10_k_7_mean.mnc']

## Create relative cluster maps

In [55]:
df_cluster_map_metadata  = pd.read_csv(metadata_cluster_map)
df_cluster_map_params = pd.DataFrame(cluster_map_params, index = [0])
df_cluster_map_params = pd.merge(df_cluster_map_metadata, df_cluster_map_params, how = 'inner')
cluster_id = str(df_cluster_map_params['id'].values[0])

cluster_map_basedir_rel = os.path.join(cluster_map_basedir, 'relative')

cluster_map_dir_rel = utils.mkdir_from_params(params = cluster_map_params,
                                              basedir = cluster_map_basedir_rel,
                                              params_id = cluster_id)

In [56]:
cluster_maps_rel = processing.create_cluster_maps(clusters = cluster_file,
                                                  imgdir = es_dir_rel_1mm,
                                                  outdir = cluster_map_dir_rel,
                                                  method = 'mean',
                                                  mask = mask_1mm)

cluster_maps_rel = glob(cluster_map_dir_rel+'*.mnc')
cluster_maps_rel[:5]

Importing cluster information...
Creating cluster maps...
nk = 2 : k = 1
nk = 2 : k = 2
nk = 3 : k = 1
nk = 3 : k = 2
nk = 3 : k = 3
nk = 4 : k = 1
nk = 4 : k = 2
nk = 4 : k = 3
nk = 4 : k = 4
nk = 5 : k = 1
nk = 5 : k = 2
nk = 5 : k = 3
nk = 5 : k = 4
nk = 5 : k = 5
nk = 6 : k = 1
nk = 6 : k = 2
nk = 6 : k = 3
nk = 6 : k = 4
nk = 6 : k = 5
nk = 6 : k = 6
nk = 7 : k = 1
nk = 7 : k = 2
nk = 7 : k = 3
nk = 7 : k = 4
nk = 7 : k = 5
nk = 7 : k = 6
nk = 7 : k = 7
nk = 8 : k = 1
nk = 8 : k = 2
nk = 8 : k = 3
nk = 8 : k = 4
nk = 8 : k = 5
nk = 8 : k = 6
nk = 8 : k = 7
nk = 8 : k = 8
nk = 9 : k = 1
nk = 9 : k = 2
nk = 9 : k = 3
nk = 9 : k = 4
nk = 9 : k = 5
nk = 9 : k = 6
nk = 9 : k = 7
nk = 9 : k = 8
nk = 9 : k = 9
nk = 10 : k = 1
nk = 10 : k = 2
nk = 10 : k = 3
nk = 10 : k = 4
nk = 10 : k = 5
nk = 10 : k = 6
nk = 10 : k = 7
nk = 10 : k = 8
nk = 10 : k = 9
nk = 10 : k = 10


['data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_7_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_10_k_5_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_6_k_2_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_9_k_3_mean.mnc',
 'data/human/derivatives/POND_SickKids/cluster_maps/relative/380-581/cluster_map_nk_4_k_2_mean.mnc']

In [24]:
# cluster_metadata = os.path.join(clustering_basedir, 'metadata.csv')
# df_cluster_metadata = pd.read_csv(cluster_metadata)
# df_cluster_metadata = df_cluster_metadata[(df_cluster_metadata['nk_max'] == nk_max) & 
#                                           (df_cluster_metadata['metric'] == metric) & 
#                                           (df_cluster_metadata['K'] == K) &
#                                           (df_cluster_metadata['sigma'] == sigma) & 
#                                           (df_cluster_metadata['t'] == t)] 

---